In [10]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pandas as pd
import isodate

In [11]:
with open('api_key.txt', 'r') as file:
    api_key = file.read().strip()

# Crea una instancia de la API de YouTube
youtube = build('youtube', 'v3', developerKey=api_key)


In [12]:
def obtener_id_canal_por_nombre(nombre_canal):
    busqueda_request = youtube.search().list(
        q=nombre_canal,
        part='id',
        maxResults=1,
        type='channel'
    )
    busqueda_response = busqueda_request.execute()
    if 'items' in busqueda_response:
        for item in busqueda_response['items']:
            if item['id']['kind'] == 'youtube#channel':
                return item['id']['channelId']
    return None

# Nombre del canal del que quieres obtener el ID
nombre_canal = 'No Somos TV'
canal_id = obtener_id_canal_por_nombre(nombre_canal)
print("ID del canal:", canal_id)


ID del canal: UCZFRsDLdgYLUIbQBSsdyVGg


In [16]:
def obtener_videos_canal(canal_id):
    videos = []
    nextPageToken = None
    fecha_hace_seis_meses = (datetime.now() - timedelta(days=1000)).isoformat() + 'Z'
    while True:
        videos_request = youtube.search().list(
            part='snippet',
            channelId=canal_id,
            maxResults=50,  # Puedes ajustar este valor según tus necesidades
            type='video',
            order='date',
            publishedAfter=fecha_hace_seis_meses,
            pageToken=nextPageToken
        )
        videos_response = videos_request.execute()
        for item in videos_response['items']:
            video_id = item['id']['videoId']
            video_info = obtener_info_video(video_id)
            if video_info:
                videos.append(video_info)
        nextPageToken = videos_response.get('nextPageToken')
        if not nextPageToken:
            break
    return videos

def obtener_info_video(video_id):
    video_request = youtube.videos().list(
        part='snippet,statistics,contentDetails',
        id=video_id
    )
    video_response = video_request.execute()
    if video_response['items']:
        video_info = video_response['items'][0]
        titulo = video_info['snippet']['title']
        fecha_publicacion = video_info['snippet']['publishedAt']
        likes = video_info['statistics'].get('likeCount', 0)
        #dislikes = video_info['statistics'].get('dislikeCount', 0)
        #compartidos = video_info['statistics'].get('shareCount', 0)
        vistas = video_info['statistics'].get('viewCount', 0)
        comentarios = video_info['statistics'].get('commentCount', 0)
        descripcion = video_info['snippet'].get('description', '')
        duracion = obtener_duracion_video(video_info['contentDetails']['duration'])
        return {
            'Titulo': titulo,
            'ID del Video': video_id,
            'Fecha de Publicación': fecha_publicacion,
            'Likes': likes,
            #'Dislikes': dislikes,
            #'Compartidos': compartidos,
            'Vistas': vistas,
            'n_Comentarios': comentarios,
            'Descripción': descripcion,
            'Duración': duracion
        }
    else:
        return None

def obtener_duracion_video(duracion_iso8601):
    duracion = isodate.parse_duration(duracion_iso8601)
    horas = duracion.days * 24 + duracion.seconds // 3600
    minutos = (duracion.seconds % 3600) // 60
    segundos = duracion.seconds % 60
    return '{:02}:{:02}:{:02}'.format(horas, minutos, segundos)


def obtener_comentarios_video(video_id):
    comentarios = []
    nextPageToken = None
    while True:
        comentarios_request = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,  # Puedes ajustar este valor según tus necesidades
            pageToken=nextPageToken
        )
        comentarios_response = comentarios_request.execute()
        for item in comentarios_response['items']:
            comentario = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comentarios.append(comentario)
        nextPageToken = comentarios_response.get('nextPageToken')
        if not nextPageToken:
            break
    return comentarios

In [17]:
# ID del canal del que quieres obtener los videos
canal_id = 'UCZFRsDLdgYLUIbQBSsdyVGg'
videos_canal = obtener_videos_canal(canal_id)

# Agregar la información de los comentarios para cada video
for video in videos_canal:
    video_id = video['ID del Video']
    comentarios = obtener_comentarios_video(video_id)
    video['Comentarios'] = comentarios

# Crear un DataFrame a partir de la lista de diccionarios
df_videos = pd.DataFrame(videos_canal)

# Guardar el DataFrame en un archivo CSV
df_videos.to_csv('videos_canal_all.csv', index=False)


In [18]:
df_videos

,Titulo,ID del Video,Fecha de Publicación,Likes,Vistas,n_Comentarios,Descripción,Duración,Comentarios
0,"""PASTRULO BUSCA A SU JULIETA"" - SUPER MATCH",-88xo7JV2TU,2024-04-22T00:00:07Z,1297,18264,69,¡Le gustan los pirañas que chapan y matan 😂! E...,00:18:15,[Que rico formato 😂 debería ser más veces a la...
1,¡BUEENAAA... MI QUERIDO MOLLEJITA! #humor #com...,CvG3_1IW9Ro,2024-04-21T17:00:08Z,176,3235,5,MIRA EL PROGRAMA COMPLETO 👉🏼 https://youtu.be/...,00:00:33,"[😂😂😂, Mollejita de codorniz, jajajaja saludos,..."
2,¡CHOLO SOY DEJÓ SU GRIPE CONERA EN #NOSOMOSTV!...,T8tJL6Ei7p0,2024-04-21T15:00:31Z,484,9775,7,MIRA EL PROGRAMA COMPLETO 👉🏼 https://youtu.be/...,00:00:25,"[Cacash lo maximo,bendiciones a todos.saludos ..."
3,¡A OSCAR TITO LE DICEN ÚLTIMO NIVEL DE MARIO B...,zF-qdPmwzwY,2024-04-21T02:00:33Z,426,8732,8,MIRA EL PROGRAMA COMPLETO 👉🏼 https://youtu.be/...,00:00:31,"[Poper siempre tan iceberg, Cacash es un cacar..."
4,¡EL PAJARILLO DE POPER ESTÁ IGUAL QUE SU PODCA...,zpaTUiYE4RQ,2024-04-21T01:00:33Z,99,1834,0,MIRA EL PROGRAMA COMPLETO 👉🏼 https://youtu.be/...,00:00:29,[]
...,...,...,...,...,...,...,...,...,...
496,"CHAPA TU MONEY - Programa 8 ""Ricardo discúlpat...",iNhodqolAmI,2022-07-28T02:00:11Z,26337,1247505,630,"Este programa está que quema, Jorge y Ricardo ...",01:15:26,"[Debe volver la tonta Queen &lt;3, Mónica torr..."
497,CLIP - NOCHE DE PATAS “NDP NOTICIAS” 🎥,JGrqh6mAW1Q,2022-07-26T23:06:00Z,885,31609,9,Estrenamos nuevo segmento en Noche de Patas: N...,00:16:45,"[El loco, debe ser fijo. Al menos solo debe sa..."
498,"CLÍPTORIS - CHAPA TU MONEY ""Ese mono está mamado""",w-9ymMMe9kY,2022-07-26T19:07:06Z,1130,54619,14,Uno de los juegos favoritos de estos últimos p...,00:10:15,"[Sigan los éxitos, ya van dos auspiciadores, q..."
499,"CLÍPTORIS - CHAPA TU MONEY ""Muy poca carne par...",-8-oiPG16ak,2022-07-25T19:16:38Z,1858,74248,17,Duelo de 8 viene a Chapa tu Money a ponerle fl...,00:18:24,"[Buen juego,pero a Maricielo le falta mas pala..."


In [19]:
# Encontrar la última fecha de publicación
ultima_fecha = df_videos['Fecha de Publicación'].min()

print("La última fecha de publicación es:", ultima_fecha)

La última fecha de publicación es: 2022-07-25T19:16:38Z
